In [1]:
#needed libraries
import scipy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import sklearn
import statsmodels
import keras
import tensorflow

In [2]:
!pip install tensorflow

In [6]:
pip install pandas-datareader

Note: you may need to restart the kernel to use updated packages.


In [7]:
# steps to build a ML model 
# 1. problem defination
# 2. loading packages
# 3. exploratiry data analysis -> visualization, stats
# 4. data preparation -> data cleaning, feature selection, data transformation 
# 5. evaluate models -> train-test split, identify evaluation metrics, model comparison
# 6. model tuning and enhancement -> #grid search
# 7. finaliza model -> performance on test data, model / variable intuition, save/deploy model

In [ ]:
# Model Selection Process:
    # 1. Simplicity
    # 2. Training Time
    # 3. Handling Non-Linearity
    # 4. Robustness to overfitting
    # 5. Dataset size
    # 6. Number of features 
    # 7. Model Interpretation
    # 8. Feature Scaling

In [8]:
# step 1:
from pandas.plotting import scatter_matrix
import pandas_datareader.data as web
import datetime as dt
import scipy as sc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
import statsmodels
import keras
import tensorflow
import yfinance as yf

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures

from sklearn.model_selection import train_test_split
#save model using pickle
from pickle import dump
from pickle import load

In [9]:
# Getting my data
start = dt.datetime(2018,1,2)
end = dt.datetime(2022,12,29)

stk_tickers = ['AAPL','MSFT','AMZN','TSLA','GOOG']
ccy_tickers = ['DEXJPUS','DEXUSUK']
idx_tickers = ['SP500','DJIA','VIXCLS']

yf.pdr_override()
stk_data = yf.download(stk_tickers,start,end)
ccy_data = web.get_data_fred(ccy_tickers,start,end)
idx_data = web.get_data_fred(idx_tickers,start,end)

stk_data.head()

[*********************100%%**********************]  5 of 5 completed


Adj Close                                                  Close  \
                 AAPL       AMZN       GOOG       MSFT       TSLA       AAPL   
Date                                                                           
2018-01-02  40.776520  59.450500  53.250000  80.391838  21.368668  43.064999   
2018-01-03  40.769424  60.209999  54.124001  80.765961  21.150000  43.057499   
2018-01-04  40.958797  60.479500  54.320000  81.476845  20.974667  43.257500   
2018-01-05  41.425129  61.457001  55.111500  82.486992  21.105333  43.750000   
2018-01-08  41.271267  62.343498  55.347000  82.571175  22.427334  43.587502   

                                                        ...       Open  \
                 AMZN       GOOG       MSFT       TSLA  ...       AAPL   
Date                                                    ...              
2018-01-02  59.450500  53.250000  85.949997  21.368668  ...  42.540001   
2018-01-03  60.209999  54.124001  86.349998  21.150000  ...  43.132500   
2018-01-04  60.479500  54.320000  87.110001  20.974667  ...  43.134998   
2018-01-05  61.457001  55.111500  88.190002  21.105333  ...  43.360001   
2018-01-08  62.343498  55.347000  88.279999  22.427334  ...  43.587502   

                                                           Volume            \
                 AMZN       GOOG       MSFT       TSLA       AAPL      AMZN   
Date                                                                          
2018-01-02  58.599998  52.417000  86.129997  20.799999  102223600  53890000   
2018-01-03  59.415001  53.215500  86.059998  21.400000  118071600  62176000   
2018-01-04  60.250000  54.400002  86.589996  20.858000   89738400  60442000   
2018-01-05  60.875500  54.700001  87.660004  21.108000   94640000  70894000   
2018-01-08  61.799999  55.111500  88.199997  21.066668   82271200  85590000   

                                           
                GOOG      MSFT       TSLA  
Date                                       
2018-01-02  24752000  22483800   65283000  
2018-01-03  28604000  26061400   67822500  
2018-01-04  20092000  21912000  149194500  
2018-01-05  25582000  23407100   68868000  
2018-01-08  20952000  22113000  147891000  

[5 rows x 30 columns]

In [10]:
stk_data.describe().T

count          mean           std           min  \
Adj Close AAPL  1257.0  9.635211e+01  4.649583e+01  3.416383e+01   
          AMZN  1257.0  1.198692e+02  3.559179e+01  5.945050e+01   
          GOOG  1257.0  8.596194e+01  3.126965e+01  4.881100e+01   
          MSFT  1257.0  1.887661e+02  7.505658e+01  7.951265e+01   
          TSLA  1257.0  1.317997e+02  1.179290e+02  1.193133e+01   
Close     AAPL  1257.0  9.804250e+01  4.632948e+01  3.554750e+01   
          AMZN  1257.0  1.198692e+02  3.559179e+01  5.945050e+01   
          GOOG  1257.0  8.596194e+01  3.126965e+01  4.881100e+01   
          MSFT  1257.0  1.937994e+02  7.464854e+01  8.501000e+01   
          TSLA  1257.0  1.317997e+02  1.179290e+02  1.193133e+01   
High      AAPL  1257.0  9.916848e+01  4.691575e+01  3.643000e+01   
          AMZN  1257.0  1.213915e+02  3.603749e+01  5.950000e+01   
          GOOG  1257.0  8.690094e+01  3.160843e+01  5.017700e+01   
          MSFT  1257.0  1.958064e+02  7.541554e+01  8.631000e+01   
          TSLA  1257.0  1.349707e+02  1.207890e+02  1.244533e+01   
Low       AAPL  1257.0  9.683265e+01  4.570808e+01  3.550000e+01   
          AMZN  1257.0  1.183413e+02  3.522516e+01  5.852550e+01   
          GOOG  1257.0  8.501034e+01  3.096076e+01  4.850550e+01   
          MSFT  1257.0  1.916085e+02  7.380754e+01  8.383000e+01   
          TSLA  1257.0  1.285561e+02  1.150650e+02  1.179933e+01   
Open      AAPL  1257.0  9.798050e+01  4.632365e+01  3.599500e+01   
          AMZN  1257.0  1.199539e+02  3.565338e+01  5.860000e+01   
          GOOG  1257.0  8.593290e+01  3.130506e+01  4.869500e+01   
          MSFT  1257.0  1.937734e+02  7.464738e+01  8.606000e+01   
          TSLA  1257.0  1.319332e+02  1.181179e+02  1.207333e+01   
Volume    AAPL  1257.0  1.169450e+08  5.506465e+07  3.519590e+07   
          AMZN  1257.0  8.660063e+07  4.069839e+07  1.762600e+07   
          GOOG  1257.0  3.099530e+07  1.350092e+07  6.936000e+06   
          MSFT  1257.0  3.023154e+07  1.287752e+07  8.989200e+06   
          TSLA  1257.0  1.323416e+08  9.020984e+07  2.940180e+07   

                         25%           50%           75%           max  
Adj Close AAPL  4.929744e+01  8.942937e+01  1.415702e+02  1.801910e+02  
          AMZN  8.899950e+01  1.080000e+02  1.581005e+02  1.865705e+02  
          GOOG  5.841950e+01  7.372250e+01  1.133635e+02  1.507090e+02  
          MSFT  1.140590e+02  1.954478e+02  2.515304e+02  3.376211e+02  
          TSLA  2.107533e+01  7.464200e+01  2.361633e+02  4.099700e+02  
Close     AAPL  5.100500e+01  9.121000e+01  1.429000e+02  1.820100e+02  
          AMZN  8.899950e+01  1.080000e+02  1.581005e+02  1.865705e+02  
          GOOG  5.841950e+01  7.372250e+01  1.133635e+02  1.507090e+02  
          MSFT  1.193600e+02  2.013000e+02  2.553500e+02  3.431100e+02  
          TSLA  2.107533e+01  7.464200e+01  2.361633e+02  4.099700e+02  
High      AAPL  5.143000e+01  9.261750e+01  1.445000e+02  1.829400e+02  
          AMZN  8.978250e+01  1.107805e+02  1.599375e+02  1.886540e+02  
          GOOG  5.892225e+01  7.441050e+01  1.148100e+02  1.521000e+02  
          MSFT  1.200200e+02  2.036500e+02  2.591900e+02  3.496700e+02  
          TSLA  2.147400e+01  7.568867e+01  2.399967e+02  4.144967e+02  
Low       AAPL  5.056250e+01  9.056750e+01  1.410400e+02  1.791200e+02  
          AMZN  8.801100e+01  1.063200e+02  1.562500e+02  1.848395e+02  
          GOOG  5.787600e+01  7.294950e+01  1.121420e+02  1.498875e+02  
          MSFT  1.185200e+02  1.975100e+02  2.529500e+02  3.422000e+02  
          TSLA  2.064133e+01  7.203333e+01  2.313700e+02  4.056667e+02  
Open      AAPL  5.091250e+01  9.125000e+01  1.427700e+02  1.826300e+02  
          AMZN  8.915000e+01  1.081035e+02  1.583500e+02  1.872000e+02  
          GOOG  5.850000e+01  7.351950e+01  1.133035e+02  1.518635e+02  
          MSFT  1.193900e+02  2.000500e+02  2.561600e+02  3.446200e+02  
          TSLA  2.105333e+01  7.220000e+01  2.356667e+02  4.114700e+02  
Volume    

In [11]:
#data-cleaning:
# stk_data.isnull().sum()
#dropping any 'NA' values within data
stk_data.dropna(axis=0) #on the row
#or
# stk_data.fillna(0)

Adj Close                                                \
                  AAPL       AMZN       GOOG        MSFT        TSLA   
Date                                                                   
2018-01-02   40.776520  59.450500  53.250000   80.391838   21.368668   
2018-01-03   40.769424  60.209999  54.124001   80.765961   21.150000   
2018-01-04   40.958797  60.479500  54.320000   81.476845   20.974667   
2018-01-05   41.425129  61.457001  55.111500   82.486992   21.105333   
2018-01-08   41.271267  62.343498  55.347000   82.571175   22.427334   
...                ...        ...        ...         ...         ...   
2022-12-21  134.874557  86.769997  90.250000  242.773453  137.570007   
2022-12-22  131.668243  83.790001  88.260002  236.575745  125.349998   
2022-12-23  131.299820  85.250000  89.809998  237.112091  123.150002   
2022-12-27  129.477585  83.040001  87.930000  235.354095  109.099998   
2022-12-28  125.504539  81.820000  86.459999  232.940552  112.709999   

                 Close                                                ...  \
                  AAPL       AMZN       GOOG        MSFT        TSLA  ...   
Date                                                                  ...   
2018-01-02   43.064999  59.450500  53.250000   85.949997   21.368668  ...   
2018-01-03   43.057499  60.209999  54.124001   86.349998   21.150000  ...   
2018-01-04   43.257500  60.479500  54.320000   87.110001   20.974667  ...   
2018-01-05   43.750000  61.457001  55.111500   88.190002   21.105333  ...   
2018-01-08   43.587502  62.343498  55.347000   88.279999   22.427334  ...   
...                ...        ...        ...         ...         ...  ...   
2022-12-21  135.449997  86.769997  90.250000  244.429993  137.570007  ...   
2022-12-22  132.229996  83.790001  88.260002  238.190002  125.349998  ...   
2022-12-23  131.860001  85.250000  89.809998  238.729996  123.150002  ...   
2022-12-27  130.029999  83.040001  87.930000  236.960007  109.099998  ...   
2022-12-28  126.040001  81.820000  86.459999  234.529999  112.709999  ...   

                  Open                                                \
                  AAPL       AMZN       GOOG        MSFT        TSLA   
Date                                                                   
2018-01-02   42.540001  58.599998  52.417000   86.129997   20.799999   
2018-01-03   43.132500  59.415001  53.215500   86.059998   21.400000   
2018-01-04   43.134998  60.250000  54.400002   86.589996   20.858000   
2018-01-05   43.360001  60.875500  54.700001   87.660004   21.108000   
2018-01-08   43.587502  61.799999  55.111500   88.199997   21.066668   
...                ...        ...        ...         ...         ...   
2022-12-21  132.979996  86.180000  89.730003  241.690002  139.339996   
2022-12-22  134.350006  85.519997  88.930000  241.259995  136.000000   
2022-12-23  130.919998  83.250000  87.620003  236.110001  126.370003   
2022-12-27  131.380005  84.970001  89.309998  238.699997  117.500000   
2022-12-28  129.669998  82.800003  87.500000  236.889999  110.349998   

               Volume                                           
                 AAPL      AMZN      GOOG      MSFT       TSLA  
Date                                                            
2018-01-02  102223600  53890000  24752000  22483800   65283000  
2018-01-03  118071600  62176000  28604000  26061400   67822500  
2018-01-04   89738400  60442000  20092000  21912000  149194500  
2018-01-05   94640000  70894000  25582000  23407100   68868000  
2018-01-08   82271200  85590000  20952000  22113000  147891000  
...               ...       ...       ...       ...        ...  
2022-12-21   85928000  59267200  20336400  23690600  145417400  
2022-12-22   77852100  81431300  23656100  28651700  210090300  
2022-12-23   63814900  57433700  17815000  21207000  166989700  
2022-12-27   69007800  57284000  15470900  16688600  208643400  
2022-12-28   85438400  58228600  17879600  17457100  221070500  

[1257

In [ ]:
#Feature Selection

In [12]:
#Data Transformation - Rescaling, Standardization, Normalization 
#Rescaling:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

# Initialize the MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))

# Fit and transform the scaler on stk_data
rescaled_data = scaler.fit_transform(stk_data)

# Create a new DataFrame with the rescaled data
rescaledX = pd.DataFrame(rescaled_data, columns=stk_data.columns)

# Rename the columns by adding '_res' suffix to each column name
rescaledX.columns = [f'{col}_res' for col in rescaledX.columns]

# Display the first few rows of the rescaled DataFrame
rescaledX.head()


,"('Adj Close', 'AAPL')_res","('Adj Close', 'AMZN')_res","('Adj Close', 'GOOG')_res","('Adj Close', 'MSFT')_res","('Adj Close', 'TSLA')_res","('Close', 'AAPL')_res","('Close', 'AMZN')_res","('Close', 'GOOG')_res","('Close', 'MSFT')_res","('Close', 'TSLA')_res",...,"('Open', 'AAPL')_res","('Open', 'AMZN')_res","('Open', 'GOOG')_res","('Open', 'MSFT')_res","('Open', 'TSLA')_res","('Volume', 'AAPL')_res","('Volume', 'AMZN')_res","('Volume', 'GOOG')_res","('Volume', 'MSFT')_res","('Volume', 'TSLA')_res"
0,0.045284,0.000000,0.043563,0.003406,0.023710,0.051327,0.000000,0.043563,0.003642,0.023710,...,0.044635,0.000000,0.036077,0.000271,0.021850,0.171289,0.123465,0.152008,0.131973,0.040558
1,0.045235,0.005975,0.052140,0.004856,0.023160,0.051276,0.005975,0.052140,0.005192,0.023160,...,0.048675,0.006337,0.043817,0.000000,0.023352,0.211788,0.151675,0.184874,0.166961,0.043429
2,0.046532,0.008095,0.054064,0.007610,0.022720,0.052641,0.008095,0.054064,0.008136,0.022720,...,0.048692,0.012830,0.055298,0.002050,0.021995,0.139383,0.145771,0.112249,0.126381,0.135408
3,0.049726,0.015784,0.061831,0.011524,0.023048,0.056004,0.015784,0.061831,0.012321,0.023048,...,0.050227,0.017694,0.058206,0.006188,0.022621,0.151909,0.181356,0.159090,0.141002,0.044611
4,0.048672,0.022758,0.064143,0.011850,0.026369,0.054895,0.022758,0.064143,0.012669,0.026369,...,0.051778,0.024883,0.062194,0.008277,0.022517,0.120301,0.231390,0.119586,0.128346,0.133934


In [13]:
#standardization -> transforms attributes to a standard normal distribution with a mean of zero and a standard deviation 1
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Initialize the StandardScaler and fit it on stk_data
scaler = StandardScaler().fit(stk_data)

# Transform stk_data using the fitted scaler
standardized_data = scaler.transform(stk_data)

# Create a new DataFrame with the standardized data
StandardisedX = pd.DataFrame(standardized_data, columns=stk_data.columns)

# Reset the column index to remove MultiIndex
StandardisedX.columns = StandardisedX.columns.droplevel() + '_std'

# Display the first few rows of the standardized DataFrame
StandardisedX.head()



,AAPL_std,AMZN_std,GOOG_std,MSFT_std,TSLA_std,AAPL_std,AMZN_std,GOOG_std,MSFT_std,TSLA_std,...,AAPL_std,AMZN_std,GOOG_std,MSFT_std,TSLA_std,AAPL_std,AMZN_std,GOOG_std,MSFT_std,TSLA_std
0,-1.195757,-1.698222,-1.046541,-1.444475,-0.936792,-1.187136,-1.698222,-1.046541,-1.445337,-0.936792,...,-1.197284,-1.721528,-1.071049,-1.442600,-0.941241,-0.267455,-0.804053,-0.462620,-0.601888,-0.743659
1,-1.195909,-1.676875,-1.018579,-1.439488,-0.938647,-1.187298,-1.676875,-1.018579,-1.439977,-0.938647,...,-1.184488,-1.698660,-1.045531,-1.443538,-0.936159,0.020467,-0.600376,-0.177192,-0.323960,-0.715496
2,-1.191835,-1.669300,-1.012309,-1.430013,-0.940135,-1.182979,-1.669300,-1.012309,-1.429792,-0.940135,...,-1.184434,-1.675231,-1.007679,-1.436435,-0.940750,-0.494282,-0.642999,-0.807919,-0.646309,0.186893
3,-1.181801,-1.641824,-0.986987,-1.416550,-0.939026,-1.172344,-1.641824,-0.986987,-1.415318,-0.939026,...,-1.179575,-1.657680,-0.998092,-1.422095,-0.938632,-0.405231,-0.386081,-0.401118,-0.530161,-0.703902
4,-1.185112,-1.616907,-0.979452,-1.415428,-0.927812,-1.175853,-1.616907,-0.979452,-1.414112,-0.927812,...,-1.174662,-1.631739,-0.984942,-1.414858,-0.938982,-0.629944,-0.024842,-0.744194,-0.630694,0.172437


In [14]:
#Normalization -> rescaling each observation (row) to have a length of one (called a unit norm or a vector)

In [18]:
#Creating NN in python
# step 1: process the data and split into a training and test set
# step 2: build the artificial neural network structure
# step 3: train the artificial neural network
# step 4: use the model to predict

#we first import the dependencies 

import numpy as np
import pandas as pd

from keras.layers import Dense
from keras.models import Sequential

from sklearn.metrics import confusion_matrix 

from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Use Pandas to read the CSV file into a DataFrame
dataset = pd.read_csv("downloads/Churn_Modelling.csv")

# we would want to create input array X and output target array Y
# X: we wont to create input vector of useful variables, lets take indewx 3 to 12
# Y: Our output will be column 13 - whether they are excited

X = dataset.iloc[:, 3:13].values
Y = dataset.iloc[:,13].values

# We have some columns that are not numerical, like Gender and Geography is slices 1 and 2
# We have to change these numerical value for interpretation by ANN. This is called Label Encoding

#Geography Column:
labelencoder_X_1 = LabelEncoder()
X[:,1] = labelencoder_X_1.fit_transform(X[:,1])

#Gender Column:
labelencoder_X_2 = LabelEncoder()
X[:,2] = labelencoder_X_2.fit_transform(X[:,2])

#Splitting dataset to Training Set and Testing Set
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.2, random_state = 0)

#Feature Scaling - initializing our ANN inputs for all features to be represented in the same way 
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [19]:
#Building the ANN structure
#Initializing the ANN:
model = Sequential()

#Adding the input layer and the first hidden layer, uniform distribution of the randomisation of our weights
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 10))
#Second hidden layer:
model.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
#Adding the output layer
model.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
#Compiling the ANN
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#since our output is binary, we use binary_crossentropy loss function and accuracy metric

#Train the ANN: fit the ANN to the Training Set
model.fit(X_train, y_train, batch_size = 10, epochs = 100)

Epoch 1/100
800/800 [==============================] - 1s 431us/step - loss: 0.4836 - accuracy: 0.7950
Epoch 2/100
800/800 [==============================] - 0s 416us/step - loss: 0.4339 - accuracy: 0.7960
Epoch 3/100
800/800 [==============================] - 0s 441us/step - loss: 0.4275 - accuracy: 0.7960
Epoch 4/100
800/800 [==============================] - 0s 440us/step - loss: 0.4225 - accuracy: 0.8184
Epoch 5/100
800/800 [==============================] - 0s 438us/step - loss: 0.4206 - accuracy: 0.8253
Epoch 6/100
800/800 [==============================] - 0s 457us/step - loss: 0.4188 - accuracy: 0.8282
Epoch 7/100
800/800 [==============================] - 0s 456us/step - loss: 0.4173 - accuracy: 0.8307
Epoch 8/100
800/800 [==============================] - 0s 460us/step - loss: 0.4166 - accuracy: 0.8309
Epoch 9/100
800/800 [==============================] - 0s 453us/step - loss: 0.4149 - accuracy: 0.8313
Epoch 10/100
800/800 [==============================] - 0s 466us/step - l

In [22]:
#Using our model to predict out of sample results:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5) #means yes the customer did leave
y_pred = pd.DataFrame(y_pred) #convert this scores to a dataset
print(y_pred.describe()) #showing the prediction rows

scores = model.evaluate(X_test, y_test) #get the scores of the sample space
print("%s: %2f%%" % (model.metrics_names[1], scores[1]*100)) 
#we had a 84% accuracy that is in line with our in sample data

63/63 [==============================] - 0s 371us/step
            0
count    2000
unique      2
top     False
freq     1834
63/63 [==============================] - 0s 427us/step - loss: 0.3983 - accuracy: 0.8395
accuracy: 83.950001%


In [23]:
#one thing we can do with a binary classification problem is to make the confusion matrix to represent the data
cm = confusion_matrix(y_test, y_pred)
df = pd.DataFrame(cm)
df.columns = ['Predicted No Exit', 'Predicted Exit']
df.index = ['Actual No Exit', 'Actual Exit']

df

,Predicted No Exit,Predicted Exit
Actual No Exit,1554,41
Actual Exit,280,125


In [27]:
#Supervised Learning, implementation of Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold

# Define the number of splits for cross-validation
n_splits = 5  # You can adjust this number based on your needs

# Create a KFold object
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)

model = LinearRegression()
model.fit(X,Y)

#we use RSS (sum of squared residuals) as our loss function 
#parameters that minimize loss - line of best fit
#grid search - to create a grid of all possible hyperparamter combinations and train the model using each of them
# create a model object and define a dictionary where keys are hyperparameters and values are list of parameters
    # - for linear regression this is a fit_intercept - yes/no
param_grid = {'fit_intercept': [True,False]}

#we use GridSearchCV object and provide:
# the estimator object
# paramter grid
# scoring method
# cross validation choice

# Cross validation is a resampling procedure used to evaluate machine learning models, 
# and scoring paranter is the evaluation metrics of the model

grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = 'r2', cv = kfold)
grid_result = grid.fit(X,Y)

In [29]:
# The above may lead to overfitting, so we introduce L1 and L2 regualizers to prevent overftting

#Now the cost function = RSS + linear sum of all values of coefficients 

from sklearn.linear_model import Lasso #L1
model = Lasso()
model.fit(X,Y)

from sklearn.linear_model import Ridge #L2
model = Ridge() 
model.fit(X,Y)

# The combination of both L1 and L2 is called the Elastric Net
from sklearn.linear_model import ElasticNet
model = ElasticNet()
model.fit(X,Y) #on top of lamda variable for the L1 and L2 function, we also use alpha where alpha = 0 corresponds to L2
#alpha = 1 corresponds to lasso

#effectively it is the best of both worlds, this will shrink some coefficients and set some to 0 for sparse selection

ElasticNet()

In [30]:
#Logistic Regression

#Logistic Regression model arises from the desire to model the probabilities of the output classes given
#a fucntion that is linear in X, at the same time ensuring that output probabilities sum up to 1 and remain between
# 0 and 1, this is applied through the signmoid function

# An example would to use Maximum likelihood estimation (MLE) to predict values close to 1 for the default class and
# zero for other classes

from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X,Y)

LogisticRegression()

In [ ]:
# Support Vector Machines, SVM, maxmises the margin, or the distrance between separating hyperplane / decision boundaries
# and the training samples closest to this hyperplane (support vectors)

# The margin is calculated as the perpendicular distance from the line to only the closest points, and therefore SVM
# calculates a maximum-margin boundary that achieves complete partition of all data points

# As data is always messy, we use a tuning paramter called C, the larger C is, the more violations of the hyperplane are
# permitted

from sklearn.svm import SVR
model = SVR()
model.fit(X,Y)

from sklearn.svm import SVC
model = SVC()
model.fit(X,Y)

# The hyperparamters are kernels (increasing the inputs to higher dimensions to ensure good fitting) and penalty, C

In [31]:
# K-Means Clustering is an unsupervised machine learning algorithm used for clustering 
# or grouping data points into clusters based on their similarity. Its primary objectives are:

# Objective:

# Group Similar Data: K-Means aims to partition a dataset into clusters in such a way that data points 
# within the same cluster are more similar to each other than to data points in other clusters. 
# It tries to find the underlying structure or patterns in the data.

# How It Works:

# Initialization: The algorithm starts by randomly selecting K initial cluster centroids. 
# These centroids are typically data points from the dataset or generated randomly.

# Assignment: Each data point is assigned to the nearest centroid based on a distance metric, often Euclidean distance. 
# This step forms initial clusters.

# Update Centroids: For each cluster, the centroid is updated by calculating the mean (average) 
# of all data points assigned to that cluster. These updated centroids represent the "center" of the current clusters.

# Repeat: Steps 2 and 3 are repeated iteratively until one of the stopping conditions is met. 
# Common stopping conditions include a maximum number of iterations or until centroids no longer change significantly.

from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
model.fit(X,Y)

from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X,Y)
# The hyperparamters are no. of neighbors and distance metrix

KNeighborsClassifier()

In [33]:
# Ensemble Models - to combine different classifiers into a meta-classifier that has better generealization performance 
# than each individual classifier alone

# The two most popular ensemble methods are bagging and boosting:

# Bagging or Bootstrap Aggregation is an ensemble technique of training individual models in a parallel way. Each Model
# is trained by a random subset of the data

# Boosting, trains several individual model in a sequential way. This is done by building a model from the training data
# and then creating a second model that attempts to correct the errors of the first model. Models are added until the
# training set is predicted perfectly or a maximum number of models is added


# By combining individual models, the ensemble model tends to be more flexible (less bias) and less data-sensitive / lower
# variance. Ensemble models combine multiple, simpler algorithms to obtain better performance 


# What are some methods? 
# 1. Random Forest - a tweaked version of bagged decision trees

# steps for bagging
# 1. Create many random subsamples of our dataset
# 2. Train a CART model on each sample
# 3. Given a new dataset, calculate the average prediction from each model and aggregate the prediction by each tree to  
# assign the final label by majority vote 

# One problem with decision trees like CART is that they are greedy - they choose the variable to split by using a greedy 
# algorithm that minimzies error

# Another problem is that even after bagging, the decision trees can have a lot of structural similarities and result in
# high correlation in their prediction. It would work better if predictions from the submodels ar uncorrelated or at 
# best lowly correlated 

# In CART, when selecting a split point, the learning algorithm is allowed to look through all variables and all variables
# values in order to select the most optimal split point

# The random forest algorithm changes this procedure such that each subtree can access only a random sample of features 
# when selecting a split point

# Regression:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()
model.fit(X,Y)

# Classification:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X,Y)
# hyperparameters are max no. of features and no. of estimators 

RandomForestClassifier()

In [34]:
# Adaptive Boosting (Adaboost)
# This is a boosting technique in which the basic idea to to try predictors sequentially, and each subsequent model
# attempts to fix the errors of its predecessor. At each iteration, the AdaBoost algorithm changes the sample distribution
# by modifying the weights attached to each of the instances 
    # - It increases the weights of the wrongly predicted instances and decreases the ones of the correctly 
    #   predicted instances 
#Steps:
    # 1. Initially, all observations are given equal weights 
    # 2. A model is built on a subset of data, and the predictions are made on the whole dataset. Errors are calculated. 
    # 3. While creating the next model, higher weights are given the wrongly predicted instances and decreases 
    #    the ones of the correctly predicted instances
    # 4. This process is repeated until the error function does not change, or until the maximum limit of the number of 
    #    estimators is reached
from sklearn.ensemble import AdaBoostRegressor
model = AdaBoostRegressor()
model.fit(X,Y)

from sklearn.ensemble import AdaBoostClassifier
model = AdaBoostClassifier()
model.fit(X,Y)
# hyperparamters are Learning rate and number of estimators 

AdaBoostClassifier()

In [35]:
# Gradient Boosting Method works by sequentially adding the previous underfitted predictions to the ensemble,
# ensuring the errors made previously are corrected 

# The objective is to find the best split in the data to minimize the error
# Steps:
    # 1. A model is built on a subset of data. Using this model, predictions are made on the whole of the dataset
    # 2. Errors are calculated and the loss function is used
    # 3. A new model is created using the errors of the previous step as the target varaible. The predictions made by 
    #    this new model are combined with the predictions of the previous.
    # 4. This process is repeated until the error fucntion does not change or until the maximum limit of the number
    #     of the number of estimators is reached
from sklearn.ensemble import GradientBoostingRegressor
model = GradientBoostingRegressor()
model.fit(X,Y)

from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier()
model.fit(X,Y)
# hyperparamters are same as AdaBoost

GradientBoostingClassifier()

In [37]:
# For neural networks, 
from sklearn.neural_network import MLPRegressor
model = MLPRegressor()
model.fit(X,Y)

from sklearn.neural_network import MLPClassifier
model = MLPClassifier()
model.fit(X,Y)
# hyperparamters are hidden layers and activation function

MLPClassifier()

In [ ]:
# Overview of Time Series Models: 
#     A time series is a sequence of numbers that are ordered by a time index
# A time series can be broken down into the following componenets:
#     1. Trend Component
#         - Consistent directional movement in a time series where it can be deterministic / stochastic
#     2. Seasonal Component
#         - Many time series contain seasonal variation like business cycles / climate data
#         - This is particularly true regarding commodity prices such as natural gas with depends on annual temperature
# This can be written as Y = S + T + R (S - seasonality, T - trend, R - error / remainder that is not captured)

# Autocorrelation
#     - It is the similarity between observations as a function of the time lag between them 
# Such relationships can be modeled using an autoregression model - regression of the variable against itself
# In an autoregression model, we forecast the variable of interest using a linear combination of past values of the variable

# Stationary 
#     - A time series is said to be stationary if its statistical properties do not change over time. 
#     - Thus a time series with trend or with seasonality is not stationary, as the trend and seasonality will affect
#       the value of the time series at different times 
#     - On the other hand, a white noise series is stationary, as it does not matter when you observe it; it should look 
#       similar at anyt point in time 
#     - Predicting future values using a stationary plot would be easier and most statistical models require the series
#       to be stationary to make it effective and precise predictions -> convert any non-stationary data to stationary
#     - Thus we use differencing to remove its series dependence on time

In [42]:
# Traditional Time Series Model (Including the ARIMA Model):
#     Most of the time series models aim at incorporating the trend, seasonality, and remainder components while addressing
#     the autocorrelation and stationarity embedded in the time series
    
#     For instance, the autoregressive (AR) model address the autocorrelation in the time series
    
# ARIMA:
#     If we combine stationarity with autoregression and a moving average model, 
#     we obtain a ARIMA Model (Autoregressive Integrated Moving Average
from statsmodels.tsa.arima.model import ARIMA

# Specify the order of the ARIMA model (p, d, q)
# You need to provide the appropriate values for p, d, and q
# For example, order=(1, 0, 0) corresponds to an AR(1) model
model = ARIMA(y_train, order=(1, 0, 0))

# Fit the ARIMA model to your training data
model_fit = model.fit()

# Make predictions using the fitted model
y_pred = model_fit.forecast(steps=len(y_test))

In [ ]:
# To convert a time series model to our supervised machine learning model:
#     we would offset the data - by using a previous timestep to input and the next timestep as output into our model
#     this can be done using the shift function in pandas, resembling something like a sliding window
#     we'll delete the first and last row as it does not contain any data in terms of X and Y
#      from here we will continue with the notebook 'Regression-Master Template'